In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [4]:
labels=pd.read_csv("/home/aman/dogs/labels.csv")

In [5]:
path = "/home/aman/dogs/train"

train_dir = os.listdir( path )

img_size = 100

X_train = []
y_train = []

X_test = []
y_test = []

i=0
for img_name in tqdm(train_dir):
    
    image = cv2.imread(path+"/"+img_name,1)
    
    rows,cols,height = image.shape
    
    image = cv2.resize(image,(img_size,img_size))
    
    label = labels[labels["id"]==img_name.split(".")[0]]["breed"].values[0]
    
    if (i < 9000): 
        X_train.append(image.reshape(1,img_size,img_size,3))
        
        y_train.append(label)
        
    if(i%2==0 and i < 9000):
        M = cv2.getRotationMatrix2D((cols/2,rows/2),100*np.random.randn(),1)
        
        rot = cv2.warpAffine(image,M,(cols,rows))
        
        rot = cv2.resize(rot,(img_size,img_size))
        
        X_train.append(rot.reshape(1,img_size,img_size,3))
        
        y_train.append(label)
        
    if(i%2!=0 and i < 9000):
        flp=cv2.flip(image,1)
        
        flp = cv2.resize(flp,(img_size,img_size))
        
        X_train.append(flp.reshape(1,img_size,img_size,3))
        
        y_train.append(label)
        
    i+=1
    if(i>9000):
        X_test.append(image.reshape(1,img_size,img_size,3))
        y_test.append(label)
        
X_train = np.concatenate(X_train,axis=0)

X_test = np.concatenate(X_test,axis=0)

100%|██████████| 10222/10222 [00:42<00:00, 241.78it/s]


In [6]:
list_classes=list(np.unique(labels["breed"].values))

y_tr = np.zeros((18000,120))

y_te = np.zeros((1222,120))

for index,value in enumerate(y_train):
    clas=list_classes.index(value)
    y_tr[index][clas]=1
    
    
for index,value in enumerate(y_test):
    y_te[index][clas]=1

In [7]:
np.save("X_train_" + str(img_size)+".npy",X_train)
np.save("X_test_" + str(img_size)+".npy",X_test)
np.save("y_train_" + str(img_size)+".npy",y_tr)
np.save("y_test_" + str(img_size)+".npy",y_te)